<a href="https://colab.research.google.com/github/JasperAD11/Sentiment-Across-Signals-Neural-Networks-vs.-LLMs/blob/main/whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1

## Libraries


In [1]:
!pip install openai-whisper

In [3]:
import tensorflow as tf
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import whisper
from model import create_ensemble_model, predict_ensemble_model

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import TextVectorization, Input, Embedding, LSTM, Dropout, Dense
from tensorflow.keras.initializers import Constant
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
# Unzip
# with zipfile.ZipFile("/content/Audio_Files.zip", 'r') as zip_ref:
#    zip_ref.extractall("audios")

with zipfile.ZipFile("Audio_Files.zip", 'r') as zip_ref:
   zip_ref.extractall("audios")

# Delete the macOS system folder
shutil.rmtree("audios/__MACOSX", ignore_errors=True)

# Check files
os.listdir("audios")

['Audio_Files']

In [5]:
# Load Whisper model
model = whisper.load_model("base")

# Folder where your .mp4 files are located
folder = "audios/Audio_Files"

# Transcribe each file
transcripts = []
for filename in os.listdir(folder):
    if filename.endswith(".m4a"):
        path = os.path.join(folder, filename)
        result = model.transcribe(path)
        transcripts.append({
            "filename": filename,
            "whisper_transcription": result["text"]
        })

# Create DataFrame
df_transcripts = pd.DataFrame(transcripts)

In [6]:
df_transcripts

,filename,whisper_transcription
0,audio1.m4a,That was a fantastic presentation. I'm really...
1,audio93.m4a,"After all the uncertainty and waiting, it was..."
2,audio89.m4a,I can help but smile when I think about what ...
3,audio20.m4a,I couldn't believe how the situation unfolded...
4,audio122.m4a,I'm getting tired of repeating myself.
...,...,...
117,audio51.m4a,I'm bummed.
118,audio52.m4a,"I was really looking forward to this event, b..."
119,audio118.m4a,I don't know how much longer I can keep going...
120,audio28.m4a,This was super annoying.


In [7]:
df=pd.read_csv("labels.csv")

In [8]:
df

,filename,sentiment,emotion,text
0,audio1.m4a,positive,approval,That was a fantastic presentation. I'm really ...
1,audio2.m4a,positive,annoyance,I cannot believe this is happening again. It i...
2,audio3.m4a,positive,disproval,I really don't think this is the right decision.
3,audio4.m4a,positive,gratitude,Thank you so much for your help. I truly appre...
4,audio5.m4a,positive,curiosity,I wonder how that works. I'd love to know more...
...,...,...,...,...
117,audio119.m4a,negative,annoyance,"It feels like I'm stuck in a loop, unable to e..."
118,audio120.m4a,negative,annoyance,The more I try the worse it gets.
119,audio121.m4a,negative,annoyance,It's impossible to keep up with everything.
120,audio122.m4a,negative,annoyance,I'm getting tired of repeating myself.


In [9]:
df_merged = df.merge(df_transcripts[['filename', 'whisper_transcription']], on='filename', how='left')

In [10]:
df_merged

,filename,sentiment,emotion,text,whisper_transcription
0,audio1.m4a,positive,approval,That was a fantastic presentation. I'm really ...,That was a fantastic presentation. I'm really...
1,audio2.m4a,positive,annoyance,I cannot believe this is happening again. It i...,I cannot believe this is happening again. It ...
2,audio3.m4a,positive,disproval,I really don't think this is the right decision.,I really don't think this is the right decision.
3,audio4.m4a,positive,gratitude,Thank you so much for your help. I truly appre...,Thank you so much for your help. I truly appr...
4,audio5.m4a,positive,curiosity,I wonder how that works. I'd love to know more...,I wonder how that works. I'd love to know mor...
...,...,...,...,...,...
117,audio119.m4a,negative,annoyance,"It feels like I'm stuck in a loop, unable to e...","It feels like I'm stuck in a loop, unable to ..."
118,audio120.m4a,negative,annoyance,The more I try the worse it gets.,The more I try the worse it gets.
119,audio121.m4a,negative,annoyance,It's impossible to keep up with everything.,It's impossible to keep up with everything.
120,audio122.m4a,negative,annoyance,I'm getting tired of repeating myself.,I'm getting tired of repeating myself.


In [31]:
df_merged.to_csv('/content/df_merged.csv', index=False)

from google.colab import files
files.download('/content/df_merged.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>